# Trends-based recommendation
In this notebook we will classify the tweets into trends and these trends will help us decide which hashtags to recommend 

In [1]:
from files_reader import *
import tensorflow as tf

## Get the data

In [2]:
tweets_and_trends = []
tweets = []
trends = []

# tweets_and_trends += (FilesReader.read_file(UK_tweets_file))
# tweets_and_trends += (FilesReader.read_file(US_tweets_file))
# tweets_and_trends += (FilesReader.read_file(AUS_tweets_file))
# tweets_and_trends += (FilesReader.read_file(IR_tweets_file))
# tweets_and_trends += (FilesReader.read_file(CAN_tweets_file))
tweets_and_trends += (FilesReader.read_file(new_US_file))
tweets_and_trends += (FilesReader.read_file(new_UK_file))
tweets_and_trends += (FilesReader.read_file(new_AUS_file))
tweets_and_trends += (FilesReader.read_file(new_CAN_file))
tweets_and_trends += (FilesReader.read_file(new_IR_file))
# tweets_and_trends += (FilesReader.read_file(new_SINGA_file))   
# tweets_and_trends += (FilesReader.read_file(new_SA_file))

random.shuffle(tweets_and_trends)

tweets, trends = FilesReader.split_tweets_and_trends(tweets_and_trends)
print(f"We have {len(tweets)} tweets.")


We have 29013 tweets.


## Stemming the words
i.e turning words like playing, plays played to play

In [3]:
# from nltk.stem import PorterStemmer


# def stem_tweet(tweet):
#     ps = PorterStemmer()
#     new_tweet = ""
#     for word in tweet.split(' '):
#         new_tweet += ps.stem(word) + ' '
#     return new_tweet


In [4]:
# print(f"before: {tweets[:3]}")
# for i, tweet in enumerate(tweets):
#     tweets[i] = stem_tweet(tweet)

# print(f"after: {tweets[:3]}")

## Tokenize the text

In [5]:
from keras.preprocessing.text import Tokenizer

tweets_tokenizer = Tokenizer(oov_token="<OOV>")
tweets_tokenizer.fit_on_texts(tweets)
tweets_word_index = tweets_tokenizer.word_index
tweets_index_word = tweets_tokenizer.index_word

print(f"We have {len(tweets_word_index)} different words")
print(tweets_word_index)

We have 22654 different words
{'<OOV>': 1, 'may': 2, 'day': 3, 'happy': 4, 'amp': 5, 'new': 6, 'today': 7, 'workers': 8, 'monday': 9, 'game': 10, 'month': 11, 'time': 12, 'us': 13, 'get': 14, 'one': 15, 'bank': 16, 'like': 17, 'good': 18, 'holiday': 19, 'first': 20, 'week': 21, 'mental': 22, 'bruins': 23, 'lets': 24, 'celebrate': 25, 'health': 26, 'love': 27, 'go': 28, 'luca': 29, 'great': 30, 'brecel': 31, 'im': 32, 'dont': 33, 'see': 34, 'win': 35, 'back': 36, 'take': 37, 'everyone': 38, 'well': 39, 'st': 40, 'know': 41, 'team': 42, 'year': 43, 'make': 44, 'panthers': 45, 'th': 46, 'going': 47, 'work': 48, 'fans': 49, 'last': 50, 'tonight': 51, 'people': 52, 'start': 53, 'next': 54, 'need': 55, 'season': 56, 'join': 57, 'labour': 58, 'morning': 59, 'every': 60, 'got': 61, 'cant': 62, 'boston': 63, 'would': 64, 'want': 65, 'best': 66, 'goal': 67, 'way': 68, 'world': 69, 'come': 70, 'still': 71, 'live': 72, 'right': 73, 'play': 74, 'free': 75, 'years': 76, 'hope': 77, 'look': 78, 'anot

## Create the padded sequences

In [6]:
from keras.utils import pad_sequences
sequence_length = 15

tweets_sequences = tweets_tokenizer.texts_to_sequences(tweets)
tweets_sequences_padded = pad_sequences(tweets_sequences, padding="post", maxlen=sequence_length)

## Map the trends to numbers

In [7]:
trends_map = {}

counter = 0

for trend in trends:
    if not (trend in trends_map):
        trends_map[trend] = counter
        counter += 1

no_of_trends = len(trends_map)
inv_trends_map = {v: k for k, v in trends_map.items()}
print(f"We have {no_of_trends} different trends")
print(trends_map)

We have 56 different trends
{'aflcrowspies': 0, 'FLAvsBOS': 1, 'MayDay': 2, 'MasterChef': 3, 'Labour Day': 4, 'MentalHealthAwarenessMonth': 5, 'LIVTOT': 6, 'MondayMotivation': 7, 'NationalPhysiciansDay': 8, 'BankHolidayMonday': 9, 'Flames': 10, 'rufc': 11, 'Charlie': 12, 'Klopp': 13, 'New Month': 14, 'Jock Zonfrillo': 15, 'Happy Beltane': 16, 'nrldragonsbulldogs': 17, 'Truro': 18, 'MondayMood': 19, 'Bank Holiday Monday': 20, 'Bruins': 21, 'MentalHealthWeek': 22, 'Andrew Dillon': 23, 'MetGala': 24, 'JP Morgan': 25, 'AAPIHeritageMonth': 26, 'AccommodationsInFilmsAndSongs': 27, 'Sutter': 28, 'Luca Brecel': 29, 'Patino': 30, 'The Stand': 31, 'Panthers': 32, 'Healy': 33, 'Snoop': 34, 'Luton': 35, 'May Day': 36, 'wunvsyd': 37, 'Scotland': 38, '4corners': 39, 'Richarlison': 40, 'Bealtaine': 41, 'Bergeron': 42, 'Latham': 43, 'Chevron': 44, 'Keith Stroud': 45, 'First Republic': 46, 'Joanna Cherry': 47, 'Mitch Love': 48, 'Jota': 49, 'Steph': 50, 'Skipp': 51, 'Tony Abbott': 52, 'Dalymount': 53, '

## Create the trends sequences

In [8]:
trends_sequences = [trends_map[trend] for trend in trends]
print(trends_sequences)

[0, 1, 2, 3, 1, 2, 4, 1, 2, 5, 6, 2, 2, 7, 8, 9, 10, 2, 1, 5, 11, 12, 13, 0, 1, 14, 2, 15, 16, 9, 2, 10, 11, 2, 11, 4, 2, 17, 18, 2, 19, 20, 19, 7, 1, 2, 2, 7, 21, 0, 1, 2, 7, 0, 22, 23, 9, 17, 11, 15, 24, 1, 12, 19, 9, 6, 25, 10, 0, 0, 19, 5, 7, 17, 6, 19, 1, 19, 9, 26, 2, 24, 24, 2, 6, 11, 5, 9, 2, 2, 1, 27, 28, 29, 9, 30, 1, 26, 1, 0, 31, 17, 32, 5, 10, 26, 33, 33, 10, 11, 12, 34, 2, 2, 11, 1, 2, 4, 25, 7, 35, 0, 2, 20, 1, 24, 17, 36, 5, 10, 10, 11, 5, 26, 2, 1, 9, 33, 22, 2, 2, 26, 19, 2, 10, 0, 0, 2, 29, 5, 2, 18, 27, 0, 37, 24, 32, 24, 29, 19, 8, 2, 10, 26, 31, 1, 1, 10, 11, 7, 19, 24, 7, 0, 0, 1, 9, 0, 10, 15, 7, 10, 5, 2, 11, 7, 27, 27, 1, 24, 1, 11, 7, 7, 2, 7, 10, 9, 36, 36, 1, 2, 10, 0, 21, 7, 11, 0, 7, 31, 1, 26, 2, 19, 6, 7, 36, 2, 19, 0, 24, 29, 0, 2, 24, 2, 9, 29, 2, 28, 26, 1, 19, 5, 20, 26, 0, 11, 27, 12, 10, 11, 32, 7, 5, 1, 17, 19, 9, 17, 7, 1, 11, 38, 11, 9, 2, 2, 36, 39, 11, 11, 2, 4, 2, 34, 11, 8, 14, 1, 40, 24, 14, 20, 10, 1, 21, 20, 2, 9, 10, 2, 2, 31, 14, 7, 2,

## Encode the trends

In [9]:
from keras.utils import to_categorical
import tensorflow as tf

encoded_trends = to_categorical(trends_sequences)
print(encoded_trends.shape)

(29013, 56)


## Prepare the pre-trained embeddings

In [10]:
from Embeddings.embeddings_matrix import get_embeddings_matrix

embeddings_index_path = "./Embeddings/embeddings_index_object.pkl"
embeddings_matrix, hits, misses = get_embeddings_matrix(tweets_word_index, embeddings_index_path)

print(f"Hits: {hits}, Misses: {misses}")

Hits: 19101, Misses: 3553


## Split the data

In [11]:
training_split = 0.8
training_tweets_count = int(0.8 * len(tweets_sequences_padded))

In [12]:
train_data = tweets_sequences_padded[0:training_tweets_count]
train_labels = encoded_trends[0:training_tweets_count]
test_data = tweets_sequences_padded[training_tweets_count:]
test_labels = encoded_trends[training_tweets_count:]

print(f'we have {len(train_data)} tweets for training and {len(test_data)} for testing')

we have 23210 tweets for training and 5803 for testing


## Tune the hyper-parameters

In [13]:
# import keras_tuner as kt
# import keras
# from keras import initializers
# import tensorflow as tf

# no_of_tweets_words = len(tweets_word_index) + 1
# embedding_dimensions = 300

# def model_builder(hp):
#     model = keras.Sequential()

#     hp_conv_filters = hp.Int('conv_filters', min_value=32, max_value=256, step=16)
#     hp_kernel_size = hp.Int('conv_kernel_size', min_value=2, max_value=5, step=1)
#     hp_dropout = hp.Float('dropout', min_value=0.0, max_value=0.6, step=0.1)
#     hp_lstm = hp.Int('lstm_units', min_value=32, max_value=256, step=16)

#     model.add(keras.layers.Embedding(
#         no_of_tweets_words,
#         embedding_dimensions,
#         input_length=sequence_length,
#         embeddings_initializer=initializers.Constant(embeddings_matrix),
#         trainable=True
#     ))
#     model.add(keras.layers.Conv1D(hp_conv_filters, hp_kernel_size, padding='same'))
#     model.add(keras.layers.Dropout(hp_dropout))
#     model.add(keras.layers.Bidirectional(keras.layers.LSTM(hp_lstm)))
#     model.add(keras.layers.Dropout(hp_dropout))
#     model.add(keras.layers.Dense(no_of_trends))

#     # Tune the learning rate for the optimizer
#     # Choose an optimal value from 0.01, 0.001, or 0.0001
#     hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

#     model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
#                   loss=keras.losses.CategoricalCrossentropy(
#                       from_logits=True),
#                   metrics=['accuracy'])

#     return model


In [14]:
# tuner = kt.Hyperband(model_builder,
#                      objective='val_accuracy',
#                      max_epochs=6,
#                      factor=3,
#                      directory='parameters_tuning',
#                      project_name='trends_classifier')

In [15]:
# tuner.search(train_data, train_labels, epochs=50, validation_data=(test_data, test_labels))

In [16]:
# best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

# print(f"""
# The hyperparameter search is complete. Here are the optimal configurations:
#     conv_filters: {best_hps.get('conv_filters')}
#     conv_kernel_size: {best_hps.get('conv_kernel_size')}
#     lstm_units: {best_hps.get('lstm_units')}
#     dropout: {best_hps.get('dropout')}
#     learning_rate: {best_hps.get('learning_rate')}
# """)

SyntaxError: incomplete input (3183150782.py, line 10)

In [ ]:
# trends_classifier = tuner.hypermodel.build(best_hps)
# history = trends_classifier.fit(train_data, train_labels, epochs=20, validation_data=(test_data, test_labels))

# val_acc_per_epoch = history.history['val_accuracy']
# best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
# print('Best epoch: %d' % (best_epoch,))

In [ ]:
# trends_classifier = tuner.hypermodel.build(best_hps)
# trends_classifier.fit(train_data, train_labels, epochs=best_epoch, validation_data=(test_data, test_labels))

## Build the model
After tuning the hyper-parameters, here are the optimal configurations: <br>
    conv_filters: 224 <br>
    conv_kernel_size: 2<br>
    lstm_units: 144<br>
    dropout: 0.1<br>
    learning_rate: 0.001<br>

In [17]:
#hyperparameters
embedding_dimensions = 300
lstm_units = 144
dropout_value = 0.1
conv_filters = 224
conv_kernel_size = 2

In [18]:
from keras import initializers
import tensorflow as tf

no_of_tweets_words = len(tweets_word_index) + 1

trends_classifier = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        no_of_tweets_words,
        embedding_dimensions,
        input_length=sequence_length,
        embeddings_initializer=initializers.Constant(embeddings_matrix),
        trainable=True
    ),
    tf.keras.layers.Conv1D(conv_filters, conv_kernel_size),
    tf.keras.layers.AveragePooling1D(),
    tf.keras.layers.Dropout(dropout_value),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units)),
    tf.keras.layers.Dropout(dropout_value),
    tf.keras.layers.Dense(no_of_trends, activation='softmax')
])

trends_classifier.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

trends_classifier.summary()

# trends_classifier = tf.keras.models.load_model("trends_classifier")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 15, 300)           6796500   
                                                                 
 conv1d (Conv1D)             (None, 14, 224)           134624    
                                                                 
 average_pooling1d (AverageP  (None, 7, 224)           0         
 ooling1D)                                                       
                                                                 
 dropout (Dropout)           (None, 7, 224)            0         
                                                                 
 bidirectional (Bidirectiona  (None, 288)              425088    
 l)                                                              
                                                                 
 dropout_1 (Dropout)         (None, 288)               0

In [19]:
epochs = 4
trends_classifier.fit(train_data, train_labels, epochs=epochs, validation_data=(test_data, test_labels))

Epoch 1/4
726/726 [==============================] - 17s 15ms/step - loss: 1.8528 - accuracy: 0.4995 - val_loss: 1.4368 - val_accuracy: 0.5952
Epoch 2/4
726/726 [==============================] - 10s 14ms/step - loss: 1.0493 - accuracy: 0.7012 - val_loss: 1.3014 - val_accuracy: 0.6448
Epoch 3/4
726/726 [==============================] - 10s 14ms/step - loss: 0.6582 - accuracy: 0.8064 - val_loss: 1.3899 - val_accuracy: 0.6438
Epoch 4/4
726/726 [==============================] - 10s 14ms/step - loss: 0.4229 - accuracy: 0.8739 - val_loss: 1.5415 - val_accuracy: 0.6359


## Save the model

In [20]:
import pickle

trends_classifier.save("./trends_classifier/trends_classifier_model.h5")
with open('./trends_classifier/inv_trends_map.pkl', 'wb') as output:
    pickle.dump(inv_trends_map, output)
with open('./trends_classifier/tweet_tokenizer.pkl', 'wb') as output:
    pickle.dump(tweets_tokenizer, output)


In [21]:
import numpy as np

def predict(tweet, tweet_tokenizer, trends_map, inv_trends_map, pad_length, model):
    tweet_sequence = tweet_tokenizer.texts_to_sequences([tweet])[0]
    padded_tweet_sequence = pad_sequences([tweet_sequence], maxlen=pad_length, padding='post')
    prediction = (model.predict(padded_tweet_sequence))
    trends_indices = np.argsort(prediction, axis=-1)[0][-3:]
    return [inv_trends_map[trend_index] for trend_index in trends_indices]

In [29]:
tweet = "happy may day."

print(predict(tweet, tweets_tokenizer, trends_map, inv_trends_map, sequence_length, trends_classifier))

1/1 [==============================] - 0s 18ms/step
['Happy Beltane', 'May Day', 'MayDay']
